In [1]:
import numpy as np
import torch
import torch.nn.functional as F


def encode_onehot(labels):
    """
    From Thomas Kips repo
    """
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_MAO():
    import networkx as nx
    from gklearn.utils.graphfiles import loadDataset
    atom_to_onehot = {'C': [1., 0., 0.], 'N': [0., 1., 0.], 'O': [0., 0., 1.]}
    Gs, y = loadDataset(
        "/home/luc/TRAVAIL/DeepGED/MAO/dataset.ds")
    #t_classes = torch.Tensor(encode_onehot(y))
    max_size = 30
    adjs = []
    inputs = []
    for i, G in enumerate(Gs):
        I = torch.eye(G.order(), G.order())
        A = torch.Tensor(nx.adjacency_matrix(G).todense())
        adj = F.pad(A+I, pad=(0, max_size-G.order(), 0, max_size-G.order()))
        adjs.append(adj)

        f_0 = []
        for _, label in G.nodes(data=True):
            cur_label = atom_to_onehot[label['atom']].copy()
            f_0.append(cur_label)

        X = F.pad(torch.Tensor(f_0), pad=(0, 0, 0, max_size-G.order()))
        inputs.append(X)
    return inputs, adjs, y  # t_classes

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import gcn
#from svd import iterated_power as compute_major_axis
compute_major_axis=svd.CustomMajorAxis.apply

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.total_nb_graphs=len(GraphList)
        self.nb_feature=20
        
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print('nb_edge labels=',self.nb_edge_labels)
        self.device=torch.device("cuda:0")
        self.gcn=gcn.GCN2(self.nb_labels,self.nb_feature,dropout=0.1).to(self.device)
        self.node_weighs=nn.Parameter(torch.ones(2,requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        self.edge_weighs=nn.Parameter(torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.float,device=self.device)
        self.labels=torch.empty(self.total_nb_graphs,card_max,self.nb_labels,dtype=torch.float,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict,card_max)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,:,:]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        print('nb_edge_pair_label,nb_edge_labels',nb_edge_pair_label,self.nb_edge_labels)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_subst,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
        f_computed= torch.zeros(self.total_nb_graphs).to(self.device)    
        node_features=torch.zeros(self.total_nb_graphs,self.card.max(),self.nb_feature).to(self.device)    
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            
            n=self.card[g1]
            m=self.card[g2]
            if not f_computed[g1]:
                node_features[g1,0:n,:]=self.gcn(self.labels[g1][0:n,:],self.A[g1][0:n*n].view(n,n)+torch.eye(n,n,device=self.device))                
                f_computed[g1]=1
                #print('m1=',self.gcn(self.labels[g1][0:n,:],self.A[g1][0:n*n].view(n,n)+torch.eye(n,n,device=self.device)))
            if not f_computed[g2]:
                node_features[g2,0:m,:]=self.gcn(self.labels[g2][0:m,:],self.A[g2][0:m*m].view(m,m)+torch.eye(m,m,device=self.device))
                f_computed[g2]=1
                #print('m2=',self.gcn(self.labels[g2][0:m,:],self.A[g2][0:m*m].view(m,m)+torch.eye(m,m,device=self.device)))
        
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            
            n=self.card[g1]
            m=self.card[g2]
            m1=node_features[g1,0:n,:]
            m2=node_features[g2,0:m,:]
            C=self.construct_cost_matrix(g1,g2,m1,m2,node_subst,edge_costs,nodeInsDel,edgeInsDel)      
            S=self.mapping_from_cost(C,n,m) # FW
            #S=self.mapping_from_similarity(C,n,m) # SVD or iterated power.
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                #normalize_factor=(nodeInsDel*n+edgeInsDel*nb_edge1)*(nodeInsDel*m+edgeInsDel*nb_edge2)                                    
                normalize_factor=(nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2))
            ged[k]=.5*(v.t()@(C+torch.diag(C))@v)/normalize_factor    
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        #cn=torch.where(x<1.0,x,torch.ones_like(x))
        ce=self.edge_weighs*self.edge_weighs
        #ce=torch.where(y<1.0,y,torch.ones_like(y))
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        #node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        #upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        #node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        #node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return cn[0],cn[1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v]['label'][0] in node_labels:
                    node_labels.append(G.nodes[v]['label'][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict,nb_max_vertices):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=float,weight='bond_type').todense(),dtype=torch.int)        
        lab=torch.zeros(nb_max_vertices,self.nb_labels)
        k=0
        
        for v in nx.nodes(G):
            lab[k][dict[G.nodes[v]['label'][0]]]=1.0
            k=k+1
   
        return (A.view(1,A.shape[0]*A.shape[1]),lab)

    def construct_cost_matrix(self,g1,g2,m1,m2,node_subst,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        #l1=self.labels[g1][0:n,:]
        #l2=self.labels[g2][0:m,:]        
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        norm_l1=torch.einsum('i,j->ij',torch.diag(m1@m1.T),torch.ones(m,device=self.device))
        norm_l2=torch.einsum('i,j->ij',torch.diag(m2@m2.T),torch.ones(n,device=self.device)).T
        scalar=m1@m2.T
        
        d=torch.empty(n,m+1,device=self.device)
        x=(norm_l1+norm_l2-2*scalar)/m1.shape[1]
        eps=10**(-6)
        d[:,0:m]=node_subst*torch.sqrt(torch.where(x>eps,x,eps*torch.ones(n,m,device=self.device)))       
        d[:,-1]=nodeInsDel
       # print('d=',d)
        D[0:n*(m+1)]=d.view(1,d.shape[0]*d.shape[1])
        #D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        #pdist = nn.PairwiseDistance(p=2)
        #D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([pdist(l1[k//(m+1),:],l2[k%(m+1),:]) for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
     
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c.view(n+1,m+1)),10).view((n+1)*(m+1),1)
    
        return svd.franck_wolfe(x0,D,c,2,10,n,m)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
        
     
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def mapping_from_similarity(self,C,n,m):
        M=self.similarity_from_cost(C)
        first_ev=compute_major_axis(M)
        #first_ev=self.iterated_power(M,inv=True)
        if(first_ev.sum() <0):
            first_ev=-first_ev
        S=torch.exp(first_ev.view(n+1,m+1)) # enforce the difference, accelerate the convergence. 
        S=self.eps_assigment_from_mapping(S)
        return  S
        
    def eps_assigment_from_mapping(self,S):
        ones_n = torch.ones(S.shape[0],device=S.device)
        ones_m = torch.ones(S.shape[1],device=S.device)
    
        Sk = S
        for i in range(20):
            D=torch.diag(1.0/(Sk@ones_m))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk1 = D@Sk
            D=torch.diag(1.0/(ones_n@Sk1))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk = Sk1@D
        
        return Sk

import networkx as nx
from gklearn.utils.graphfiles import loadDataset

#from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
#writer = SummaryWriter('runs/fashion_GED_experiment_1')

Gs, y = loadDataset("/home/luc/TRAVAIL/DeepGED/MAO/dataset.ds")
model = Net(Gs,normalize=True)

params = list(model.parameters())
print(len(params))
print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

['C', 'N', 'O']
{'C': 0, 'N': 1, 'O': 2} 3
nb_edge labels= 3
torch.Size([68, 729])
adjacency matrices tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]], device='cuda:0')
node labels tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [

In [3]:
import itertools
nb=len(Gs)
class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

train_size=15

if train_size % 2 == 0:
    nb_class1=int(train_size/2)
    nb_class2=int(train_size/2)
else:
    nb_class1=int(train_size/2)+1
    nb_class2=int(train_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]
train_graphs=torch.cat((random_class1,random_class2),0)
print(train_graphs)

couples=torch.triu_indices(train_size,train_size,offset=1)

#combinations=itertools.combinations(range(nb),2)

nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)

data[0:nb_elt,0]=train_graphs[couples[0]]
data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

print('data=',data)

#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object   
    
#writer.add_graph(model,data)
#writer.close()


tensor([19, 28, 28, 28, 23, 26, 27, 17])
tensor([20,  4, 25, 18, 34, 66, 53, 36, 12, 67,  0, 39, 31, 51, 57])
105
data= tensor([[20,  4],
        [20, 25],
        [20, 18],
        [20, 34],
        [20, 66],
        [20, 53],
        [20, 36],
        [20, 12],
        [20, 67],
        [20,  0],
        [20, 39],
        [20, 31],
        [20, 51],
        [20, 57],
        [ 4, 25],
        [ 4, 18],
        [ 4, 34],
        [ 4, 66],
        [ 4, 53],
        [ 4, 36],
        [ 4, 12],
        [ 4, 67],
        [ 4,  0],
        [ 4, 39],
        [ 4, 31],
        [ 4, 51],
        [ 4, 57],
        [25, 18],
        [25, 34],
        [25, 66],
        [25, 53],
        [25, 36],
        [25, 12],
        [25, 67],
        [25,  0],
        [25, 39],
        [25, 31],
        [25, 51],
        [25, 57],
        [18, 34],
        [18, 66],
        [18, 53],
        [18, 36],
        [18, 12],
        [18, 67],
        [18,  0],
        [18, 39],
        [18, 31],
        [18, 51]

In [4]:
from triangular_losses import ReducedTriangularConstraint as triangular_constraint
model.to(device)
def classification(model,data,yt,nb_iter):
    torch.autograd.set_detect_anomaly(True)
    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0)
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    print(device)

#torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    input=data.to(device)
    target=yt.to(device) 
#torch.ones(40,device=device)
#target[20:]=-1.0
#target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    nodeSubst=np.empty(nb_iter)
    node_subst,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    


        y_pred = model(input).to(device)
    
    
        # Compute and print loss
        loss = criterion(y_pred, target).to(device)    
        node_subst,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
        triangularInq=criterionTri(node_subst,nodeInsDel,edge_costs,edgeInsDel)
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        loss_plt[t]=loss.item()
        nodeSubst[t]=node_subst.item()
        k=0
        
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        print(t, loss.item()) 
        if t % 100 == 99 or t==0:                           
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('nodeInsDel:',nodeInsDel.item())
            print('node Subst:',node_subst.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return nodeSubst,InsDel,edgeSub,loss_plt
nb_iter=100
nodeSubst,InsDel,edgeSub,loss_plt=classification(model,data,yt,nb_iter)


cuda:0
0 0.5665070414543152
Distances:  tensor([0.6399, 0.8878, 0.7011, 0.7248, 0.6648, 0.6269, 0.7417, 0.5102, 1.0706,
        0.5099, 1.0377, 0.5207, 1.7675, 1.2460, 0.9008, 0.8991, 0.8909, 0.8342,
        0.6741, 0.8458, 0.5010, 1.3704, 0.4439, 1.1853, 0.5829, 2.1328, 1.6657,
        0.7359, 0.7635, 0.6948, 0.6048, 0.7335, 0.4940, 1.1384, 0.4805, 1.0479,
        0.5438, 1.6951, 1.2808, 0.7708, 0.6871, 0.6107, 0.7394, 0.5214, 1.1163,
        0.5272, 1.0717, 0.5302, 1.8186, 1.2934, 0.7013, 0.6510, 0.8094, 0.5186,
        1.0973, 0.5404, 1.0487, 0.5512, 1.8117, 1.2699, 0.5906, 0.7324, 0.4898,
        1.0640, 0.4934, 1.0143, 0.5055, 1.5441, 1.2188, 0.8554, 0.5084, 1.2391,
        0.5072, 1.1486, 0.5277, 1.7990, 1.4331, 0.5258, 1.1335, 0.4793, 1.0845,
        0.5584, 1.8392, 1.4222, 1.5567, 0.4798, 1.4234, 0.6493, 2.3074, 1.8390,
        0.5678, 0.8961, 0.5583, 1.4448, 1.1050, 1.6175, 0.7232, 2.5672, 2.0863,
        0.5081, 1.2532, 1.0098, 2.0949, 1.6529, 0.8015], device='cuda:0',
      

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()

plt.figure(1)
plt.plot(nodeSubst,label="node subst")
plt.title('Node substitution costs')
plt.legend()

plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the loss')
plt.show()
plt.close()

In [ ]:
from __future__ import division
from __future__ import print_function

import random
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from models import GCN
from utils import load_MAO

# Model and optimizer
model = GCN(3,
            nhid=10,
            nclass=2,
            dropout=0.1)

optimizer = optim.Adam(model.parameters(), lr=0.001)

inputs, adjs, t_classes = load_MAO()  # dataset, n_batch=1)
criterion = torch.nn.CrossEntropyLoss()
min_loss = 10000
best_acc = 0

for epoch in range(10000):  # loop over the dataset multiple times
    running_loss = 0.0
    i = 0
    acc = 0
    data = list(zip(inputs, adjs, t_classes))
    random.shuffle(data)
    for X, adj, y in data:

        # get the inputs; data is a list of [inputs, labels]
        label = torch.tensor([y]).long()
        # zero the parameter gradients

        p = torch.randperm(30)
        X = X[p, :]
        adj = adj[p, :][:, p]

        # forward + backward + optimize
        outputs = model(X, adj)
        outputs = outputs.reshape(1, -1)
        loss = criterion(outputs, label)
        pred = outputs.argmax().item()
        # print(pred)
        if (pred == y):
            acc = acc + 1

        if (i % 10 == 0):
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        i = i + 1
        # print statistics
        running_loss += loss.item()
    # if epoch == 3000:
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = 0.00001

    #optimizer = adjust_learning_rate(optimizer, epoch)
    cur_loss = running_loss/len(inputs)
    if (cur_loss < min_loss):
        min_loss = cur_loss
        PATH = './weights/optimal_net.pth'
        torch.save(model.state_dict(), PATH)
    cur_acc = acc/len(t_classes)
    if (cur_acc > best_acc):
        best_acc = cur_acc
    if epoch % 1000 ==0:
        print(f"Epoch {epoch}, loss: {cur_loss}, acc : {acc/len(t_classes)}")

print(
    f"Finished Training, best acc achieved : {best_acc}, best loss : {min_loss}")

PATH = './weights/my_net.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
#print (data[0])
model2 = GCN2(3,
            nhid=10,
            nclass=2,
            dropout=0.1)
X,adj,y=data[0]
output=model2(X,adj)
print(len(data))
print(adj)
print('X=',X)
print('output=',output)
#print(model(X,adj),model(X,adj).reshape(1,-1),y)